This notebook is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/).

Author: 蘇嘉冠

# 自然語言處理基礎功（一）

請記得先複製一份在你自己的 Google 帳號底下：
`檔案` -> `在雲端硬碟中儲存副本`

也請記得改變 colab 設定：
`工具` -> `設定` -> `編輯器` -> 將 `縮排寬度` 改為 `4`

## 練習題：我就懶

對於常常使用社群網路的人來說，每天接受到的資訊量可以說是非常大量。雖然便利，但同時也讓人的閱讀習慣改變，對於長篇的文章內容難有耐心消化吸收。因此這次練習我們試著讓機器閱讀大量文章，並且從中抽取最重要的關鍵字，讓世界上的懶人可以快速掌握世界脈動。

練習用的資料來自衛生福利部疾病管制署在 2020 年 5 月 18 日 至 28 日的新聞稿（[連結](https://www.cdc.gov.tw/Bulletin/List/MmgtpeidAR5Ooai4-fgHzQ)），總共 30 篇。

![](https://i.imgur.com/Ohdj6bJ.png)

（[圖片來源](https://memes.tw/wtf/373564)）

In [ ]:
!pip install pandas jieba wordcloud

In [ ]:
import jieba
import jieba.analyse
import jieba.posseg
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

### Step 1. 斷詞

首先，我們必須要先了解如何做斷詞，因此選用 jieba 這個工具來小小測試一下。

In [ ]:
sentence = "說到台中美食，我最喜歡的就是大甲鎮瀾宮的中部粽了！"

words = jieba.lcut(sentence)
print(words)

雖然上面的斷詞結果整體還不錯，但因為 jiaba 主要還是用簡體中文的詞彙來做當作字典，因此有些台灣的用語可能會切不好（例如`台中`就被切開了），因此我們使用專門為台灣繁體中文的字典（[原始來源](https://github.com/ldkrsi/jieba-zh_TW)），讓斷詞的結果可以更精確。

In [ ]:
!wget https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/nlp_tw/dict.txt -O dict.txt

In [ ]:
jieba.set_dictionary("dict.txt")

words = jieba.lcut(sentence)
print(words)

除了斷詞，jieba 也可以同時做詞性標記（POS Tagging）。詞性符號的對應表可以參考[這裡](https://docs.google.com/spreadsheets/d/1KIzjDozuH5EooDofcf-kIyVPcpikYZfyLueDa_PFVSk/edit#gid=0)。

In [ ]:
pos_words = jieba.posseg.lcut(sentence)
print(pos_words)

我們也可以試試把台語（漢字版本）的當作輸入試試（取自[滅火器 島嶼天光](https://www.youtube.com/watch?v=VZMqcZF7wic)），可以看到`袂當`被切斷了。這時候我們還可以做的把自定義的字詞加到字典裡面。


In [ ]:
sentence = "親愛的媽媽，請你毋通煩惱我，原諒我，行袂開跤，我欲去對抗袂當原諒的人。"

words = jieba.lcut(sentence)
print(words)

In [ ]:
jieba.add_word("袂當")

words = jieba.lcut(sentence)
print(words)

### Step 2. 基於頻率的關鍵字抽取

接下來我們試著用詞出現的頻率來當作抽取關鍵字的方法。先是定義 function `count_word_freq()` 來去計算斷詞後每個詞出現的頻率，然後再用 `show_most_freq()` 秀出最常出現的 20 個詞。這裡的測試資料來自衛生福利部疾病管制署在 2020 年 5 月 28 日的[新聞稿](https://www.cdc.gov.tw/Bulletin/Detail/MykTNwPGIGQYv03z6c2rYg?typeid=9)。


In [ ]:
def count_word_freq(words):
    # Count the frequency for each word.
    word_freq = {}
    for word in words:
        if word not in word_freq:
            word_freq[word] = 1
        else:
            word_freq[word] = word_freq[word] + 1

    # Convert the format from  dict to list of tuple.
    word_freqs = [(word, freq) for word, freq in word_freq.items()]

    return word_freqs

In [ ]:
sentence = "中央流行疫情指揮中心今(28)日公佈國內新增299例COVID-19確定病例，分別為297例本土個案及2例境外移入個案；另有校正回歸本土個案258例，總計557例。確診個案中新增19例死亡。指揮中心表示，今日新增之297例本土病例，為157例男性，140例女性，年齡介於未滿5歲至90多歲，發病日介於今(2021)年4月29日至5月27日；個案分佈以新北市136例最多，其次為台北市94例，桃園市21例，台中市10例，花蓮縣9例，彰化縣7例，基隆市及台南市各6例，嘉義市2例，新竹縣、屏東縣、台東縣、南投縣、新竹市及高雄市各1例。指揮中心進一步表示，校正回歸個案258例中，為133例男性，125例女性，年齡介於未滿5歲至90多歲，發病日介於5月11日至5月27日；個案分佈以新北市141例最多，其次為台北市114例，彰化縣2例，桃園市1例。指揮中心指出，綜上所有個案，其中萬華活動史相關110例、茶藝館相關9例、某社團相關1例，其他已知感染源181例、關聯不明143例、疫調中111例。相關疫情調查持續進行中。指揮中心說明，今日新增19例死亡個案（案1800、2127、2393、2654、2813、3337、3341、4194、4366、4578、4852、5029、5170、5338、5409、6125、6399、6530、6824），共計為男性14位、女性5位，年齡介於40多歲至80多歲，發病日介於5月10日至5月24日，確診日介於5月17日至5月28日，死亡日期介於5月21日至5月27日，詳如新聞稿附件。指揮中心指出，新增2例境外移入個案中，案7332為本國籍10歲以下男性，5月18日於澳洲出現流鼻水等症狀，5月26日返臺，持有登機前3日內檢驗陰性報告，入境時主動通報曾有症狀，採檢後於今日確診。案7333為菲律賓籍20多歲女性，5月14日來臺工作，持有登機前3日內檢驗陰性報告，入境後至檢疫所集中檢疫，5月24日進行檢疫期滿前採檢，於今日確診。指揮中心統計，累計455,198例新型冠狀病毒肺炎相關通報(含397,534例排除)，其中7,315例確診，分別為1,126例境外移入，6,136例本土病例，36例惇睦艦隊、2例航空器感染、1例不明及14例調查中；另18例(案530、1589、1676、1591、1886、2067、2068、2528、2530、2599、3459、3921、3975、3989、3990、3991、4059、4726)移除為空號。確診個案中78例死亡。指揮中心再次呼籲，民眾應落實手部衛生、咳嗽禮節及佩戴口罩等個人防護措施，減少不必要移動、活動或集會，避免出入人多擁擠的場所，或高感染傳播風險場域，並主動積極配合各項防疫措施，共同嚴守社區防線。"

words = jieba.lcut(sentence)

In [ ]:
word_freqs = count_word_freq(words)
print(word_freqs)

In [ ]:
def show_most_freq(word_freqs, topk=20):
    # Sorted by word frequency.
    sorted_word_freqs = sorted(word_freqs, key=lambda wf: wf[1], reverse=True)

    # Get k top frequency words.
    topk_word_freqs = sorted_word_freqs[:topk]
    topk_words = [wf[0] for wf in topk_word_freqs]
    topk_freqs = [wf[1] for wf in topk_word_freqs]

    # Show 
    df = pd.DataFrame({
        "word": topk_words,
        "frequency": topk_freqs,
    })
    print(df)

In [ ]:
show_most_freq(word_freqs)

上面雖然可以秀出最常出現的詞，但會發現很多都是不太重要的字詞或是標點符號。比較正確的作法是將斷詞的結果做進一步的篩選，方法是將屬於停用詞（Stop Words）的字詞符號去掉。去掉停用詞後再做同樣的事情。

停用詞可以自己定義，我們在這裡用的停用詞來自[這裡](https://github.com/tomlinNTUB/Python)。

附註：如果今天資料是英文，除了移除停用詞之外，也可能會做 Lemmatization 或是 Stemming （[參考連結](https://ithelp.ithome.com.tw/articles/10214221)）。不過這裡是中文，就不會做這個步驟了。

In [ ]:
!wget https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/nlp_tw/stop_words.txt -O stop_words.txt

In [ ]:
with open("stop_words.txt", "r") as f:
    stop_words = f.readlines()
    stop_words = [stop_word.rstrip() for stop_word in stop_words]

In [ ]:
def remove_stop_words(words, stop_words):
    filtered_words = []
    for word in words:
        if word not in stop_words:
            filtered_words.append(word)

    return filtered_words

In [ ]:
filtered_words = remove_stop_words(words, stop_words)

print(filtered_words)

In [ ]:
word_freqs = count_word_freq(filtered_words)
show_most_freq(word_freqs)

練習時間：

請找一篇你有興趣的文章，將 `sentence` 內容改成文章內文，並且再跑一次看看。

### Step 3. 基於 TF-IDF 的關鍵字抽取

TF-IDF 是關鍵字抽取最常用的方法之一，核心思想是不只看詞在文章（我們這裡的 `sentence`）中出現的頻率，也考慮這個詞在不同文章中出現的頻率，綜合兩者來給詞的重要性打分數，分數越高者代表越有可能是關鍵字。可以看這部[影片](https://www.youtube.com/watch?v=vZAXpvHhQow)了解更多。

我們可以使用 jiabe 非常輕鬆的得到 TF-IDF 關鍵字的結果！

![](https://i.imgur.com/tYtkqau.png)
（[圖片來源](https://mropengate.blogspot.com/2016/04/tf-idf-in-r-language.html)）
![](https://i.imgur.com/T7fs0ha.png)
（[圖片來源](https://www.youtube.com/watch?v=vZAXpvHhQow)）

In [ ]:
jieba.analyse.set_stop_words("stop_words.txt")

In [ ]:
key_words = jieba.analyse.extract_tags(sentence, topK=20)
print(key_words)

### Step 4. 關鍵字視覺化

終於到這個步驟，我們先下載疾管署新聞稿文章到 Colab Server 的 `dcd_articles` 這個資料夾裡面。文章會是一個 `.txt` 結尾的文件檔。

In [ ]:
!wget https://github.com/SuJiaKuan/fgu_ai_course/raw/main/datasets/nlp_tw/dcd_articles.zip && unzip -qo dcd_articles.zip

In [ ]:
!ls dcd_articles

為了要讀取所有文章的內容，首先要先找到所有檔案的路徑，再將內容都讀取起來。

In [ ]:
import glob

filenames = glob.glob("dcd_articles/*.txt")
print(filenames)

In [ ]:
sentences = []
for filename in filenames:
    with open(filename, "r") as f:
        sentence = f.read()
        sentences.append(sentence)

print(sentences)

讀取所有文章後，用 TF-IDF 去抓取每個文章的關鍵字，並且將所有文章的關鍵字都放在 `all_key_words` 這個 list 之中。

In [ ]:
all_key_words = []
for sentence in sentences:
    key_words = jieba.analyse.extract_tags(sentence, topK=20)
    all_key_words = all_key_words + key_words

print(all_key_words)

最後，我們用文字雲（Word Cloud）來視覺化所有文章的關鍵字 `all_key_words`。由於我們是用中文的，必須要使用中文字型才有辦法正常顯示。

In [ ]:
!wget https://github.com/SuJiaKuan/fgu_ai_course/raw/main/datasets/nlp_tw/SourceHanSansTW-Regular.otf -O SourceHanSansTW-Regular.otf

In [ ]:
wc = WordCloud(
    background_color="white",
    font_path="SourceHanSansTW-Regular.otf",
)
wc.generate(" ".join(all_key_words))

plt.imshow(wc)
plt.axis("off")
plt.show()

練習時間：
1. 請參考 [jieba 文件](https://github.com/fxsjy/jieba#%E5%9F%BA%E4%BA%8E-tf-idf-%E7%AE%97%E6%B3%95%E7%9A%84%E5%85%B3%E9%94%AE%E8%AF%8D%E6%8A%BD%E5%8F%96)與[詞性對照表](https://docs.google.com/spreadsheets/d/1KIzjDozuH5EooDofcf-kIyVPcpikYZfyLueDa_PFVSk/edit#gid=0)，讓 Step 3 跟 Step4 的結果是只含以下詞性的關鍵字：
    - 名詞（`n`）
    - 人名（`nr`）
    - 機構團體（`nt`）
    - 地名（`ns`）
    - 其他專名（`nz`）
2. [TextRank](https://towardsdatascience.com/textrank-for-keyword-extraction-by-python-c0bae21bcec0) 是另一個常常被使用的關鍵字抽取方法，請參考 [jieba 文件](https://github.com/fxsjy/jieba#%E5%9F%BA%E4%BA%8E-textrank-%E7%AE%97%E6%B3%95%E7%9A%84%E5%85%B3%E9%94%AE%E8%AF%8D%E6%8A%BD%E5%8F%96)，將 Step3 跟 Step4 的 TF-IDF 改成 TextRank，並且重新跑看看結果有何不同。